In [ ]:
import cv2
import requests
from pygame import mixer
import time
mixer.init()
sound = mixer.Sound('beep-01a.wav')


TELEGRAM_BOT_TOKEN = 'Write your telegram bot token'
TELEGRAM_CHAT_ID = 'write your telegram chat id '



def send_telegram_message(message):
    send_text = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage?chat_id={TELEGRAM_CHAT_ID}&parse_mode=Markdown&text={message}'
    response = requests.get(send_text)
    return response.json()


cap = cv2.VideoCapture(1) 


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


square_size = 200  


x = int(frame_width / 2 - square_size / 2)
y = int(frame_height / 2 - square_size / 2)
small_square_area = (x, y, square_size, square_size)


fgbg = cv2.createBackgroundSubtractorMOG2()


cv2.namedWindow('Motion Detection', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Motion Detection', 800, 600)  

while True:
    ret, frame = cap.read()

    if not ret:
        break

    
    x, y, w, h = small_square_area
    roi = frame[y:y+h, x:x+w]

   
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  

   
    fgmask = fgbg.apply(roi)

   
    threshold = 50
    _, thresh = cv2.threshold(fgmask, threshold, 255, cv2.THRESH_BINARY)

 
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False

    for contour in contours:
        if cv2.contourArea(contour) > 100:  
            motion_detected = True

          
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2) 

            break

    if motion_detected:
        print("Motion detected in the smaller square area!")
        send_telegram_message("Ah! man Run, SomeOne is at the door")
        try:
            sound.play()
        except: 
            isplaying = False
            pass
  
    frame[y:y+h, x:x+w] = cv2.resize(roi, (w, h))

    cv2.imshow('Motion Detection', frame)

    if cv2.waitKey(30) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()